# Filtrando dígitos 1 e 5 e treinando modelos

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
train_redu = pd.read_csv("dataset_digits/train_redu.csv")
test_redu  = pd.read_csv("dataset_digits/test_redu.csv")

print("Train shape:", train_redu.shape)
print("Test shape:", test_redu.shape)
print(train_redu.head())

Train shape: (2186, 3)
Test shape: (1115, 3)
   label  intensidade   simetria
0      0    145.43529  148.57256
1      0    118.57647  137.11372
2      0    127.60000  134.04706
3      0    138.04706  151.00392
4      0    146.21568  122.50196


In [4]:
train1x5 = train_redu[train_redu["label"].isin([1, 5])].copy()
test1x5  = test_redu[test_redu["label"].isin([1, 5])].copy()

# Criação da variável-alvo y (+1 para dígito 1 e -1 para dígito 5)
train1x5["y"] = train1x5["label"].apply(lambda x: 1 if x == 1 else -1)
test1x5["y"]  = test1x5["label"].apply(lambda x: 1 if x == 1 else -1)

In [5]:
X_1x5 = train1x5[["intensidade", "simetria"]].values.astype(float)
y = train1x5["y"].values.astype(int)

X_train_1x5, X_val_1x5, y_train, y_val = train_test_split(
    X_1x5, y, test_size=0.2, random_state=42, stratify=y
)

X_test_1x5 = test1x5[["intensidade", "simetria"]].values.astype(float)
y_test = test1x5["y"].values.astype(int)

# Escalonamento dos dados (0 a 1)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_1x5)
X_val   = scaler.transform(X_val_1x5)
X_test  = scaler.transform(X_test_1x5)

print("\n--- Verificação das dimensões ---")
print("Tamanho treino:", X_train.shape)
print("Tamanho validação:", X_val.shape)
print("Tamanho teste:", X_test.shape)


--- Verificação das dimensões ---
Tamanho treino: (947, 2)
Tamanho validação: (237, 2)
Tamanho teste: (583, 2)


In [ ]:
"""class PocketPLA:
    def __init__(self):
        self.w = None

    def get_w(self):
        return self.w
    
    def set_w(self, w):
        self.w = w

    def execute(self, X, y, max_iter=1000):
        # Inicializa w com zeros
        self.w = np.zeros(X.shape[1])

        # Guarda o melhor w e o menor erro
        pocket_w = self.w.copy()
        best_error = self.errorIN(X, y)

        for _ in range(max_iter):
            atualizado = False
            for i in range(len(y)):
                if np.sign(np.dot(self.w, X[i])) != y[i]:
                    # Atualiza pesos
                    self.w += y[i] * X[i]
                    atualizado = True

                    # Calcula erro atual
                    current_error = self.errorIN(X, y)

                    # Se o novo w for melhor, guarda no pocket
                    if current_error < best_error:
                        pocket_w = self.w.copy()
                        best_error = current_error
                    break  # volta a percorrer o dataset do início

            # Se não houve atualização, já convergiu
            if not atualizado:
                break

        # Usa o melhor w encontrado
        self.w = pocket_w

    def getOriginalY(self, originalX):
        return (-self.w[0] - self.w[1]*originalX) / self.w[2]
    
    def h(self, x):
        return np.sign(np.dot(self.w, x))
    
    def errorIN(self, X, y):
        erros = 0
        for i in range(len(y)):
            if np.sign(np.dot(self.w, X[i])) != y[i]:
                erros += 1
        return erros
    
# === Adiciona o bias ao conjunto de treino, validação e teste ===
X_train_bias = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_val_bias   = np.hstack([np.ones((X_val.shape[0], 1)), X_val])
X_test_bias  = np.hstack([np.ones((X_test.shape[0], 1)), X_test])"""

### Perceptron

In [15]:
class PocketPLA:
    def __init__(self):
        self.w = None

    def get_w(self):
        return self.w
    
    def set_w(self, w):
        self.w = w

    def execute(self, X, y, max_iter=1000):
        # Adiciona bias internamente
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])

        # Inicializa w como vetor de zeros
        self.w = np.zeros(X_bias.shape[1])

        # Armazena o melhor w (pocket) e o menor erro
        pocket_w = self.w.copy()
        best_error = self.errorIN(X_bias, y)

        for _ in range(max_iter):
            for i in range(len(y)):
                if np.sign(np.dot(self.w, X_bias[i])) != y[i]:
                    # Atualiza w com erro atual
                    self.w += y[i] * X_bias[i]

                    # Calcula erro com novo w
                    current_error = self.errorIN(X_bias, y)

                    # Se for melhor, guarda no pocket
                    if current_error < best_error:
                        pocket_w = self.w.copy()
                        best_error = current_error
                    break  # volta ao início do loop principal

        # Ao final, define o melhor w encontrado
        self.w = pocket_w
    
    def getOriginalY(self, originalX):
        """Calcula Y original da fronteira de decisão (para plot 2D)."""
        return (-self.w[0] - self.w[1]*originalX) / self.w[2]

    def h(self, x):
        # Adiciona bias automaticamente ao vetor de entrada
        x_bias = np.insert(x, 0, 1)  # insere o 1 no início
        return np.sign(np.dot(self.w, x_bias))
    
    def errorIN(self, X_bias, y):
        error = 0
        for i in range(len(y)):
            if np.sign(np.dot(self.w, X_bias[i])) != y[i]:
                error += 1
        return error

In [16]:
# === Treina o modelo PocketPLA ===
perceptron = PocketPLA()
perceptron.execute(X_train, y_train)

print("\n--- Vetor de pesos treinado ---")
print(perceptron.w)

# === Predição no conjunto de teste ===
y_pred = np.array([perceptron.h(x) for x in X_test])

print("\n--- Predições no conjunto de teste ---")
print(y_pred[:20])


--- Vetor de pesos treinado ---
[ 1.         -0.92235091 -0.66832498]

--- Predições no conjunto de teste ---
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [19]:
"""class RegressaoLinear:
    def __init__(self):
        self.w = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        n_samples, n_features = X.shape

        # Adiciona bias (coluna de 1s)
        X = np.hstack([X, np.ones((n_samples, 1))])

        # Solução analítica: w = (X^T X)^-1 X^T y
        self.w = np.linalg.pinv(X.T @ X) @ (X.T @ y)

    def predict(self, X, class_output=True):
        # Adiciona bias
        X_bias = np.c_[X, np.ones(X.shape[0])]

        # Predição contínua
        y_pred = X_bias @ self.w

        # Se class_output=True, converte para classes -1 / +1
        if class_output:
            y_class = np.sign(y_pred)
            y_class[y_class == 0] = 1
            return y_class

        return y_pred
    
linreg = RegressaoLinear()
linreg.fit(X_train, y_train)

print("\n--- Regressão Linear ---")
print("Vetor de pesos treinado:", linreg.w)
y_pred_lin_class = linreg.predict(X_test)
y_pred_lin_cont = linreg.predict(X_test, class_output=False)
print("Predições (classe):", y_pred_lin_class[:10])
print("Predições (contínuas):", y_pred_lin_cont[:10])"""

'class RegressaoLinear:\n    def __init__(self):\n        self.w = None\n\n    def fit(self, X, y):\n        X = np.array(X)\n        y = np.array(y)\n\n        n_samples, n_features = X.shape\n\n        # Adiciona bias (coluna de 1s)\n        X = np.hstack([X, np.ones((n_samples, 1))])\n\n        # Solução analítica: w = (X^T X)^-1 X^T y\n        self.w = np.linalg.pinv(X.T @ X) @ (X.T @ y)\n\n    def predict(self, X, class_output=True):\n        # Adiciona bias\n        X_bias = np.c_[X, np.ones(X.shape[0])]\n\n        # Predição contínua\n        y_pred = X_bias @ self.w\n\n        # Se class_output=True, converte para classes -1 / +1\n        if class_output:\n            y_class = np.sign(y_pred)\n            y_class[y_class == 0] = 1\n            return y_class\n\n        return y_pred\n    \nlinreg = RegressaoLinear()\nlinreg.fit(X_train, y_train)\n\nprint("\n--- Regressão Linear ---")\nprint("Vetor de pesos treinado:", linreg.w)\ny_pred_lin_class = linreg.predict(X_test)\ny_pre

### Regressão Linear

In [22]:
class LinearRegression:
    def __init__(self):
        self.w = None

    def execute(self, _X, _y, show=False):
        X0 = np.asarray(_X, dtype=float)
        y = np.asarray(_y, dtype=float).ravel()

        # Garante formato 2D
        if X0.ndim == 1:
            X0 = X0.reshape(-1, 1)

        # Matriz de projeto
        X = np.c_[np.ones((X0.shape[0], 1)), X0]

        if show:
            print("X =\n", X)
            print("y =\n", y)

        # Solução via pseudo-inversa (SVD internamente)
        self.w = np.linalg.pinv(X) @ y

        if show:
            print(f"W =\n{self.w}")

    def predict(self, _x, class_output=False):
        x0 = np.asarray(_x, dtype=float)
        if x0.ndim == 1:
            x0 = x0.reshape(1, -1)

        X = np.c_[np.ones((x0.shape[0], 1)), x0]
        y_pred = X @ self.w

        if class_output:
            y_class = np.sign(y_pred)
            y_class[y_class == 0] = 1
            return y_class

        return y_pred

    def get_w(self):
        return self.w

In [24]:
print("\n=== Treinando Regressão Linear ===")
linreg = LinearRegression()
linreg.execute(X_train, y_train)

print("--- Vetor de pesos (Regressão Linear) ---")
print(linreg.get_w())

print("\nPredições (valores contínuos):", linreg.predict(X_test)[:10])
print("Predições (classe):", linreg.predict(X_test, class_output=True)[:10])


=== Treinando Regressão Linear ===
--- Vetor de pesos (Regressão Linear) ---
[ 1.56309114 -1.39341773 -1.77398181]

Predições (valores contínuos): [1.1289766  0.83943899 1.232256   0.92278039 0.98800396 1.1328054
 0.74303143 0.66856743 1.38109177 1.2276304 ]
Predições (classe): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Regressão Logística

In [25]:
class RegressaoLogistica:
    def __init__(self, eta=0.01, tmax=5000, lambda_=0.01):
        self.eta = eta
        self.tmax = tmax
        self.lambda_ = lambda_
        self.w = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        X = np.array(X)
        y = np.where(np.array(y) <= 0, -1, 1)

        X = np.c_[np.ones((X.shape[0], 1)), X]
        n_amostras, n_features = X.shape
        self.w = np.zeros(n_features)

        for _ in range(self.tmax):
            z = X @ self.w
            grad = -(1/n_amostras) * (X.T @ (y / (1 + np.exp(y * z))))
            grad[1:] += self.lambda_ * self.w[1:]
            self.w -= self.eta * grad

    def predict_prob(self, X):
        X = np.array(X)
        X = np.c_[np.ones((X.shape[0], 1)), X]
        return self.sigmoid(X @ self.w)

    def predict(self, X):
        X = np.array(X)
        X = np.c_[np.ones((X.shape[0], 1)), X]
        return np.sign(X @ self.w)

    def get_w(self):
        return self.w


In [26]:
print("\n=== Treinando Regressão Logística ===")
logreg = RegressaoLogistica(eta=0.1, tmax=5000, lambda_=0.01)
logreg.fit(X_train, y_train)
print("--- Vetor de pesos (Regressão Logística) ---")
print(logreg.get_w())
print("Predições (classe):", logreg.predict(X_test)[:10])
print("Probabilidades (sigmoid):", logreg.predict_prob(X_test)[:10])


=== Treinando Regressão Logística ===
--- Vetor de pesos (Regressão Logística) ---
[ 3.34592856 -3.28596798 -3.49240159]
Predições (classe): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Probabilidades (sigmoid): [0.91941264 0.86577698 0.93632255 0.88442099 0.89198937 0.91752953
 0.83922462 0.81609309 0.95041987 0.93523833]
